<a href="https://colab.research.google.com/github/smwalter96/PUS2020_MWalter/blob/master/Final_Project/PUS_Midterm_Air_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install libspatialindex-c4v5
!pip install pysal

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-c4v5 is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [2]:
!pip install geopandas
!apt install python3-rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [3]:
import geopandas as gp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from geopandas.tools import sjoin 
import pysal as ps

In [4]:
### Read in all PM2.5 data from drive, average across stations over time and combine
path = '/content/drive/MyDrive/air_pollution/pm25'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

air0 = pd.concat(li, axis=0, ignore_index=True)

air1 = air0.groupby('Site ID')['Daily Mean PM2.5 Concentration'].mean()
air0['pm25'] = air1
air1 = pd.DataFrame(air1)
air2 = air0.merge(air1, left_on = 'Site ID', right_on = 'Site ID')
air2 = air2.drop_duplicates(subset='Site Name', keep="first")
air2


,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration_x,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE,pm25,Daily Mean PM2.5 Concentration_y
0,01/02/2018,AQS,20130002,1,1.5,ug/m3 LC,6,NaN,1,100.0,88502,Acceptable PM2.5 AQI & Speciation Mass,NaN,NaN,2,Alaska,13,Aleutians East,55.324675,-160.508331,NaN,2.763333
120,01/01/2018,AQS,20200018,3,13.1,ug/m3 LC,53,GARDEN/TRINITY CHRISTIAN CHURCH,1,100.0,88101,PM2.5 - Local Conditions,11260.0,"Anchorage, AK",2,Alaska,20,Anchorage,61.205861,-149.824602,NaN,5.408192
474,01/01/2018,AQS,20201004,3,8.8,ug/m3 LC,37,PARKGATE/EAGLE RIVER,1,100.0,88101,PM2.5 - Local Conditions,11260.0,"Anchorage, AK",2,Alaska,20,Anchorage,61.326700,-149.569707,NaN,5.176879
820,05/31/2018,AQS,20500001,3,2.1,ug/m3 LC,9,Bethel,1,100.0,88502,Acceptable PM2.5 AQI & Speciation Mass,NaN,NaN,2,Alaska,50,Bethel,60.795830,-161.767000,NaN,3.477500
1020,01/02/2018,AQS,20680003,1,0.3,ug/m3 LC,1,Denali NP & PRES - Headquarters,1,100.0,88502,Acceptable PM2.5 AQI & Speciation Mass,NaN,NaN,2,Alaska,68,Denali,63.723200,-148.967600,NaN,0.977778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421421,01/02/2018,AQS,11011002,1,8.2,ug/m3 LC,34,"MOMS, ADEM",1,100.0,88101,PM2.5 - Local Conditions,33860.0,"Montgomery, AL",1,Alabama,101,Montgomery,32.412811,-86.263394,NaN,11.705467
421860,01/02/2018,AQS,11030011,1,7.8,ug/m3 LC,33,"DECATUR, Alabama",1,100.0,88101,PM2.5 - Local Conditions,19460.0,"Decatur, AL",1,Alabama,103,Morgan,34.530717,-86.967536,NaN,7.473650
422323,01/02/2018,AQS,11130003,1,8.2,ug/m3 LC,34,PHENIX CITY - SOUTH GIRARD SCHOOL,1,100.0,88101,PM2.5 - Local Conditions,17980.0,"Columbus, GA-AL",1,Alabama,113,Russell,32.437028,-84.999653,NaN,9.248545
422873,01/01/2018,AQS,11190003,3,8.4,ug/m3 LC,35,"Ward, Sumter Co.",1,100.0,88502,Acceptable PM2.5 AQI & Speciation Mass,NaN,NaN,1,Alabama,119,Sumter,32.362606,-88.277992,NaN,3.558708


In [5]:
air = air2[['Site ID', 'STATE', 'COUNTY', 'SITE_LATITUDE', 'SITE_LONGITUDE', 'Daily Mean PM2.5 Concentration_y']]
air = air.rename(columns={'Site ID': 'Site', 'Daily Mean PM2.5 Concentration_y': 'PM25'})
air

,Site,STATE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE,PM25
0,20130002,Alaska,Aleutians East,55.324675,-160.508331,2.763333
120,20200018,Alaska,Anchorage,61.205861,-149.824602,5.408192
474,20201004,Alaska,Anchorage,61.326700,-149.569707,5.176879
820,20500001,Alaska,Bethel,60.795830,-161.767000,3.477500
1020,20680003,Alaska,Denali,63.723200,-148.967600,0.977778
...,...,...,...,...,...,...
421421,11011002,Alabama,Montgomery,32.412811,-86.263394,11.705467
421860,11030011,Alabama,Morgan,34.530717,-86.967536,7.473650
422323,11130003,Alabama,Russell,32.437028,-84.999653,9.248545
422873,11190003,Alabama,Sumter,32.362606,-88.277992,3.558708


In [6]:
# Census state IDs
st = ['01','04','05','06','08','09','10','11','12','13','16','17','18','19',
      '20','21','22','23','24','25','26','27','28','29','30','31','32','33',
      '34','35','36','37','38','39','40','41','42','44','45','46','47','48',
      '49','50','51','53','54','55','56']

In [7]:
import urllib, os
for i in st:
    os.system('wget https://www2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_%s_tract.zip' % i)


In [8]:
### Read in and combine census tracts shapefile
import zipfile
dir_name = '/content/drive/My Drive/acs_2018'
extension = ".zip"

os.chdir(dir_name) 
for item in os.listdir(dir_name):
    if item.endswith(extension): 
        file_name = os.path.abspath(item) 
        zip_ref = zipfile.ZipFile(file_name) 
        zip_ref.extractall(dir_name) 
        zip_ref.close() 
        os.remove(file_name) 

In [9]:
from pathlib import Path
folder = Path('/content/drive/My Drive/acs_2018')

gdf = pd.concat([
    gp.read_file(shp)
    for shp in folder.glob("*.shp")
]).pipe(gp.GeoDataFrame)

In [10]:
### Convert air quality stations to geodataframe points
air_points = gp.GeoDataFrame(
    air, geometry=gp.points_from_xy(x=air.SITE_LONGITUDE, y=air.SITE_LATITUDE)
)

air_points

,Site,STATE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE,PM25,geometry
0,20130002,Alaska,Aleutians East,55.324675,-160.508331,2.763333,POINT (-160.50833 55.32467)
120,20200018,Alaska,Anchorage,61.205861,-149.824602,5.408192,POINT (-149.82460 61.20586)
474,20201004,Alaska,Anchorage,61.326700,-149.569707,5.176879,POINT (-149.56971 61.32670)
820,20500001,Alaska,Bethel,60.795830,-161.767000,3.477500,POINT (-161.76700 60.79583)
1020,20680003,Alaska,Denali,63.723200,-148.967600,0.977778,POINT (-148.96760 63.72320)
...,...,...,...,...,...,...,...
421421,11011002,Alabama,Montgomery,32.412811,-86.263394,11.705467,POINT (-86.26339 32.41281)
421860,11030011,Alabama,Morgan,34.530717,-86.967536,7.473650,POINT (-86.96754 34.53072)
422323,11130003,Alabama,Russell,32.437028,-84.999653,9.248545,POINT (-84.99965 32.43703)
422873,11190003,Alabama,Sumter,32.362606,-88.277992,3.558708,POINT (-88.27799 32.36261)


In [12]:
air_points = air_points.set_crs(4269)

In [18]:
### Spatial join to merge census tracts with the air quality station point it is within
census_points = gp.sjoin(air_points, gdf, op='within')


In [21]:
census_points = census_points.drop_duplicates(subset='Site', keep="last")
census_points

,Site,STATE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE,PM25,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,state1,county1,MATCH
4349,40011235,Arizona,Apache,35.880404,-109.438003,2.235714,POINT (-109.43800 35.88040),18700,04,001,944300,04001944300,9443,Census Tract 9443,G5020,S,2640766674,7310572,+36.3677367,-109.4071885,4.0,1.0,4Census Tract 94431
4447,40018001,Arizona,Apache,34.058475,-109.441259,2.221495,POINT (-109.44126 34.05848),19657,04,001,970502,04001970502,9705.02,Census Tract 9705.02,G5020,S,4570089361,22668769,+33.9271551,-109.5146032,4.0,1.0,4Census Tract 9705.021
4554,40031005,Arizona,Cochise,31.349200,-109.539683,5.540290,POINT (-109.53968 31.34920),20098,04,003,000800,04003000800,8,Census Tract 8,G5020,S,19081727,11859,+31.3807450,-109.5244977,4.0,3.0,4Census Tract 83
4899,40038001,Arizona,Cochise,32.009410,-109.389060,2.815517,POINT (-109.38906 32.00941),20094,04,003,000100,04003000100,1,Census Tract 1,G5020,S,3424486960,1088240,+32.1897342,-109.3988452,4.0,3.0,4Census Tract 13
5249,40070010,Arizona,Gila,33.654700,-111.107400,3.842241,POINT (-111.10740 33.65470),19604,04,007,000800,04007000800,8,Census Tract 8,G5020,S,641303063,21511752,+33.4526950,-110.9371613,4.0,7.0,4Census Tract 87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421421,11011002,Alabama,Montgomery,32.412811,-86.263394,11.705467,POINT (-86.26339 32.41281),691,01,101,002500,01101002500,25,Census Tract 25,G5020,S,4488073,37253,+32.4177882,-086.2722538,1.0,101.0,1Census Tract 25101
421860,11030011,Alabama,Morgan,34.530717,-86.967536,7.473650,POINT (-86.96754 34.53072),1193,01,103,005103,01103005103,51.03,Census Tract 51.03,G5020,S,7751686,746909,+34.5167960,-086.9711064,1.0,103.0,1Census Tract 51.03103
422323,11130003,Alabama,Russell,32.437028,-84.999653,9.248545,POINT (-84.99965 32.43703),1443,01,113,030800,01113030800,308,Census Tract 308,G5020,S,21342370,417245,+32.4345151,-084.9805005,1.0,113.0,1Census Tract 308113
422873,11190003,Alabama,Sumter,32.362606,-88.277992,3.558708,POINT (-88.27799 32.36261),1189,01,119,011600,01119011600,116,Census Tract 116,G5020,S,220321253,185406,+32.3800435,-088.3396696,1.0,119.0,1Census Tract 116119


In [22]:
from google.colab import files
census_points.to_csv('PM25.csv') 
files.download('PM25.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>